## Tutorial

This notebook demonstrates the main functionalities of the code in this repository. 
1. Methods to determine whether a given SoS form $f$ of degree $2k$ is uniquely Sum-of-Squares representable. It also allows to compute the Sum-of-Squares support, check nondegeneracy of the dual etc.  
2. An implementation of Algorithm 1 in the accompanying paper: It finds the decomposition $$ f_d = \sum_{i=1}^m \lambda_i q_i^d $$
where $f_d$ is given for $d = 2, 3$, $\lambda_i$ are positive weights and $q_1,\ldots,q_m$ are $k$-forms. It assumes that the Sum-of-Squares support of $f_2$ is spanned by polynomials that are algebraically independent up to degree $3$. 

### Ex 1: A family of quartics of quadratic length that is uniquely Sum-of-Squares representable

We consider the family
$$
    q_{ijk} = (X_i + X_j + X_k)(Y_i + Y_j + Y_k), \qquad (i,j,k \in \{1,\ldots, n\}, \quad i+j+k \equiv_n 0)
$$
of $m=\Theta(n^2)$ many quadratics and 
$$
    f_2 = \sum_{\substack{i,j,k=1, \\  i+j+k \equiv_n 0}}^n q_{ijk}^2.
$$
This notebook verifies for small values of $n$ that $f_2$ is uniquely Sum-of-Squares representable and that the supporting face of $f$ in $Σ_{2k}$ is exposed. 

In [2]:
using Pkg;
Pkg.activate("SoSupport");
using MosekTools; default_solver=Symbol(Mosek); # change to your favourite solver. MOSEK needs a licence. Uncomment to use the COSMO solver as a default.  
include("src/sosupport.jl");
include("src/random-quadratics.jl");
using LinearAlgebra # e.g. for rank 
using DataFrames;
# Random, DataFrames, CSV, StatsBase, COSMO

  Activating project at `~/Dokumente/git repositories/powers-of-forms/SoSupport`


In [3]:
n = 6;
k = 2;
ranktol = 7; # measured in digits
feas_tolerance = 1e-8
opt_tolerance = 1000;
@polyvar X[1:n] Y[1:n];
q = [(X[i] + X[j] + X[k])*(Y[i] + Y[j] + Y[k])  for i=1:n, j=1:n, k=1:n if (i+j+k)%n == 0 && i≤j≤k] 
# q = [(X[i] + X[j] + X[k])^2 - (Y[i] + Y[j] + Y[k])^2  for i=1:n, j=1:n, k=1:n if (i+j+k)%n == 0 && i≤j≤k] # alternative family that also yields a uniquely representable SOS.
m = length(q);
λ = [rand(1:m) for i=1:m]; # random weights
f₂ = sum(λ[i]*q[i]^2 for i=1:m);

# uncomment to check unique SOS representability in a neighbourhood
# ε = 0.01;
# q_noisy = q .+ ε.* [gaussian_splitvar_quadratic(X, Y) for i=1:m]
# f₂ = sum(q_noisy[i]^2 for i=1:m)

In [4]:
s = calc_sos_attributes(f₂; digits=ranktol, solver=default_solver, feas_tolerance=feas_tolerance, opt_tolerance=opt_tolerance);
df = report(s, m; transpose=false)

Row,Parameter,Value,Explanation
,Symbol,Any,String
1,m,10,No. of addends
2,n,12,No. of variables
3,k,2,degree
4,solver,Mosek,SDP solver
5,solve_time,0.0640685,"SDP solve time for (G, M_E)"
6,digits,7,Threshold for truncating eigenvalues (in digits)
7,uniquely_sos_representable,true,
8,dual_nondegenerate,true,Whether f lies on exposed face of Σ_2k
9,dim_sosupp,10,


### Ex 2: Our family has a unique Powers-of-forms decomposition

The family from `Ex 1` does also not satisfy any algebraic relations up to degree $3$. Setting 
$$
    f_3 = \sum_{\substack{i,j,k=1, \\  i+j+k \equiv_n 0}}^n q_{ijk}^3,
$$
it therefore follows that $f_2, f_3$ have a unique joint POF decomposition of (minimum) rank $m$. We are going to compute this decomposition. 

In [5]:
include("src/pof-decomp.jl");
f₃ = sum(λ[i]*q[i]^3 for i=1:m);
pof_dec = pof_decompose(f₂, f₃);
fieldnames(typeof(pof_dec)) # pof_dec is an object storing all the intermediate quantities used in the algorithm. 

(:f_2, :f_3, :k, :vars, :n, :q, :λ, :sosdata, :digits, :u, :N, :Y, :M_φ1, :M_φ2, :M_φ3, :ℓ, :g_2, :g_3)

`pof_decompose` returns a `PofDecomposition` object. Its fields `q` and `λ` are the addends and weights. We can check that `pof_dec.q` is indeed a decomposition of `f₃`, up to numerical errors.  

In [9]:
q_rounded = round.(pof_dec.q, digits=8);
λ_rounded = round.(pof_dec.λ, digits=8);
display(f₃ - sum(λ_rounded[i]*q_rounded[i]^3 for i=1:m)) # == 0, verifies that q and q_rounded are both decompositions. 
[q q_rounded] # same up to permutation

0.0

10×2 Matrix{Polynomial{true, Float64}}:
 9.0X₂Y₂                                                       …  9.0X₆Y₆
 X₁Y₁ + X₁Y₂ + X₁Y₃ + X₂Y₁ + X₂Y₂ + X₂Y₃ + X₃Y₁ + X₃Y₂ + X₃Y₃     9.0X₂Y₂
 4.0X₁Y₁ + 2.0X₁Y₄ + 2.0X₄Y₁ + X₄Y₄                               4.0X₃Y₃ + 2.0X₃Y₆ + 2.0X₆Y₃ + X₆Y₆
 9.0X₄Y₄                                                          X₁Y₁ + X₁Y₅ + X₁Y₆ + X₅Y₁ + X₅Y₅ + X₅Y₆ + X₆Y₁ + X₆Y₅ + X₆Y₆
 X₃Y₃ + X₃Y₄ + X₃Y₅ + X₄Y₃ + X₄Y₄ + X₄Y₅ + X₅Y₃ + X₅Y₄ + X₅Y₅     X₂Y₂ + 2.0X₂Y₅ + 2.0X₅Y₂ + 4.0X₅Y₅
 X₂Y₂ + 2.0X₂Y₅ + 2.0X₅Y₂ + 4.0X₅Y₅                            …  X₂Y₂ + X₂Y₄ + X₂Y₆ + X₄Y₂ + X₄Y₄ + X₄Y₆ + X₆Y₂ + X₆Y₄ + X₆Y₆
 4.0X₃Y₃ + 2.0X₃Y₆ + 2.0X₆Y₃ + X₆Y₆                               X₁Y₁ + X₁Y₂ + X₁Y₃ + X₂Y₁ + X₂Y₂ + X₂Y₃ + X₃Y₁ + X₃Y₂ + X₃Y₃
 X₂Y₂ + X₂Y₄ + X₂Y₆ + X₄Y₂ + X₄Y₄ + X₄Y₆ + X₆Y₂ + X₆Y₄ + X₆Y₆     4.0X₁Y₁ + 2.0X₁Y₄ + 2.0X₄Y₁ + X₄Y₄
 X₁Y₁ + X₁Y₅ + X₁Y₆ + X₅Y₁ + X₅Y₅ + X₅Y₆ + X₆Y₁ + X₆Y₅ + X₆Y₆     X₃Y₃ + X₃Y₄ + X₃Y₅ + X₄Y₃ + X₄Y₄ + X₄Y₅ + X₅Y₃ + X₅Y₄ + X₅Y₅
 

Since in our original solution, both `q` and `λ` had integer coefficients, we can recover them exactly by rounding. It is maybe not obvious to see that the original `q` and the computed solution `q_rounded` are equal up to permutation. For such situations, we have the convenience function `polycompare`, which sorts the output vectors by comparing leading terms and their coefficients. Note that this only makes sense after rounding away the numerical errors. 

In [59]:
sort!(q; lt=polycompare);
sort!(q_rounded; lt=polycompare);
[q q_rounded] # same up to permutation

10×2 Matrix{Polynomial{true, Float64}}:
 9.0X₆Y₆                                                       …  9.0X₆Y₆
 9.0X₄Y₄                                                          9.0X₄Y₄
 X₃Y₃ + X₃Y₄ + X₃Y₅ + X₄Y₃ + X₄Y₄ + X₄Y₅ + X₅Y₃ + X₅Y₄ + X₅Y₅     X₃Y₃ + X₃Y₄ + X₃Y₅ + X₄Y₃ + X₄Y₄ + X₄Y₅ + X₅Y₃ + X₅Y₄ + X₅Y₅
 4.0X₃Y₃ + 2.0X₃Y₆ + 2.0X₆Y₃ + X₆Y₆                               4.0X₃Y₃ + 2.0X₃Y₆ + 2.0X₆Y₃ + X₆Y₆
 X₂Y₂ + 2.0X₂Y₅ + 2.0X₅Y₂ + 4.0X₅Y₅                               X₂Y₂ + 2.0X₂Y₅ + 2.0X₅Y₂ + 4.0X₅Y₅
 X₂Y₂ + X₂Y₄ + X₂Y₆ + X₄Y₂ + X₄Y₄ + X₄Y₆ + X₆Y₂ + X₆Y₄ + X₆Y₆  …  X₂Y₂ + X₂Y₄ + X₂Y₆ + X₄Y₂ + X₄Y₄ + X₄Y₆ + X₆Y₂ + X₆Y₄ + X₆Y₆
 9.0X₂Y₂                                                          9.0X₂Y₂
 X₁Y₁ + X₁Y₅ + X₁Y₆ + X₅Y₁ + X₅Y₅ + X₅Y₆ + X₆Y₁ + X₆Y₅ + X₆Y₆     X₁Y₁ + X₁Y₅ + X₁Y₆ + X₅Y₁ + X₅Y₅ + X₅Y₆ + X₆Y₁ + X₆Y₅ + X₆Y₆
 X₁Y₁ + X₁Y₂ + X₁Y₃ + X₂Y₁ + X₂Y₂ + X₂Y₃ + X₃Y₁ + X₃Y₂ + X₃Y₃     X₁Y₁ + X₁Y₂ + X₁Y₃ + X₂Y₁ + X₂Y₂ + X₂Y₃ + X₃Y₁ + X₃Y₂ + X₃Y₃
 4.0X₁Y₁ + 2.0X₁Y₄ + 2.0X₄Y₁

## Ex 3: Jennrich's algorithm

One important tool for the present pof-decomposition algorithm is the classical result, often attributed to R. Jennrich.  

In [20]:
a_1 = [ 1; 0;  0; 0; 0; 0];
a_2 = [ 0; 1;  1; 0; 0; 0];
a_3 = [-1; 1; -1; 0; 0; 0];
λ = [1 2 3];
a = [a_1, a_2, a_3];
g_2 = sum(λ[i]*(a[i]⋅Y)^2 for i=1:length(a));
g_3 = sum(λ[i]*(a[i]⋅Y)^3 for i=1:length(a));
ℓ, λ = positive_weighted_jennrich(g_2, g_3);
ℓ_rounded = [round.(ℓ[i], digits=7) for i=1:length(a)];
λ_rounded = round.(λ, digits=7);
[ℓ_rounded λ_rounded]


3×2 Matrix{Polynomial{true, Float64}}:
 Y₁             1.0
 Y₂ + Y₃        4.0
 -Y₁ + Y₂ - Y₃  3.0

In [ ]:
[ℓ_rounded λ_rounded]
